# Ensembl genes table extraction EDA

This notebook is useful for development as well as exploratory data analysis on the extracted tables.
It is currently automically executed and saved as part of exports using `papermill`.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from ensembl_genes import ensembl_genes
from bioregistry import normalize_prefix

In [3]:
# parameters cell
species = "human"
release = "104"

In [4]:
# Parameters
species = "human"
release = "108"


In [5]:
ensg = ensembl_genes.Ensembl_Gene_Queries(release=release, species=species)
ensg.connection_url

'mysql+mysqlconnector://anonymous@ensembldb.ensembl.org:3306/homo_sapiens_core_108_38'

In [6]:
database = ensg.database
database

'homo_sapiens_core_108_38'

## Extract data

## gene attrib counts

In [7]:
ensg.run_query("gene_attrib_counts").head(15)

,attrib_type_id,code,name,description,attrib_type_count,attrib_type_examples
0,142,GeneGC,Gene GC,Percentage GC content for this gene,69292,"48.28, 46.76, 39.60, 39.25, 34.62, 35.90, 45.9..."
1,4,name,Name,Alternative/long name,66544,"RP11-974F13.3, ANKRD20A2P, RP11-718G2.5, NME3,..."
2,395,xref_id,Xref ID,ID of associated database reference,57029,"OTTHUMG00000133511, OTTHUMG00000171325, OTTHUM..."
3,538,legacy_biotype,Legacy biotype,Obsolete biotype previously assigned to this E...,20062,"processed_transcript, lincRNA, antisense, sens..."
4,536,Ensembl_Select,Ensembl Select,The Ensembl Select is a transcript identified ...,19060,"ENST00000440869, ENST00000304749, ENST00000366..."
5,380,havana_cv,Havana CV term,Controlled vocabulary terms from Havana,14023,"overlapping locus, ncRNA host, readthrough gen..."
6,54,remark,Remark,Annotation remark,4783,"snoRNA host, TAGENE_gene, Assembled from PacBi..."
7,560,artef_dupl,Artifactual duplication,Annotation on artifactual regions of the genom...,19,Artifactual duplication. Real copy of this gen...
8,382,NoTransRefError,No translations due to reference error,This gene is believed to include protein codin...,8,1
9,1,embl_acc,European Nucleotide Archive (was EMBL) accession,ENA,0,None


## genes

In [8]:
ensg.gene_df.head()

/home/runner/work/ensembl-genes/ensembl-genes/ensembl_genes/ensembl_genes.py:143: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  alt_allele_df = alt_allele_df.groupby("alt_allele_group_id").apply(


,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
0,ENSG00000000003,15,TSPAN6,HGNC,HGNC:11858,protein_coding,ensembl_havana,2008-04-29 11:17:41,2019-06-15 05:41:31,GRCh38,...,100627108,100639991,-1,True,NaN,no,tetraspanin 6,HGNC Symbol,HGNC:11858,ENSG00000000003
1,ENSG00000000005,6,TNMD,HGNC,HGNC:17757,protein_coding,ensembl_havana,2008-04-29 11:17:41,2018-11-21 17:23:49,GRCh38,...,100584936,100599885,1,True,NaN,no,tenomodulin,HGNC Symbol,HGNC:17757,ENSG00000000005
2,ENSG00000000419,14,DPM1,HGNC,HGNC:3005,protein_coding,ensembl_havana,2008-04-29 11:17:41,2020-12-11 08:28:43,GRCh38,...,50934867,50959140,-1,True,NaN,no,dolichyl-phosphate mannosyltransferase subunit...,HGNC Symbol,HGNC:3005,ENSG00000000419
3,ENSG00000000457,14,SCYL3,HGNC,HGNC:19285,protein_coding,ensembl_havana,2008-04-29 11:17:41,2018-11-21 17:23:49,GRCh38,...,169849631,169894267,-1,True,NaN,no,SCY1 like pseudokinase 3,HGNC Symbol,HGNC:19285,ENSG00000000457
4,ENSG00000000460,17,C1orf112,HGNC,HGNC:25565,protein_coding,ensembl_havana,2008-04-29 11:17:41,2018-11-21 17:23:49,GRCh38,...,169662007,169854080,1,True,NaN,no,chromosome 1 open reading frame 112,HGNC Symbol,HGNC:25565,ENSG00000000460


In [9]:
# clone-based genes no longer get a symbol and are filled with the stable ID
# https://www.ensembl.info/2021/03/15/retirement-of-clone-based-gene-names/
ensg.gene_df.query("gene_symbol == ensembl_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
1599,ENSG00000083622,8,ENSG00000083622,None,None,lncRNA,havana,2008-04-29 11:17:41,2009-05-19 09:47:17,GRCh38,...,117604791,117647415,-1,True,NaN,no,"novel transcript, antisense to CFTR",NaN,NaN,ENSG00000083622
1996,ENSG00000093100,13,ENSG00000093100,None,None,lncRNA,havana,2008-04-29 11:17:41,2012-06-07 23:07:01,GRCh38,...,17787652,17811497,-1,True,NaN,no,novel transcript,NaN,NaN,ENSG00000093100


In [10]:
# which external database the gene symbol derives from versus the ensembl source
pd.crosstab(
    ensg.gene_df.ensembl_source,
    ensg.gene_df.gene_symbol_source_db.fillna("missing (clone-based)"),
    margins=True,
)

gene_symbol_source_db,EntrezGene,HGNC,RFAM,miRBase,missing (clone-based),All
ensembl_source,,,,,,
ensembl,3,4232,1460,0,903,6598
ensembl_havana,34,21297,1,0,236,21568
ensembl_havana_tagene,0,5,0,0,0,5
havana,208,17299,0,2,18277,35786
havana_tagene,5,280,0,0,3089,3374
insdc,0,37,0,0,0,37
mirbase,0,1897,0,1,26,1924
All,250,45047,1461,3,22531,69292


In [11]:
ensg.gene_df.coord_system.value_counts().head(10)

chromosome    69238
scaffold         54
Name: coord_system, dtype: int64

In [12]:
ensg.gene_df.gene_biotype.value_counts().head(10)

protein_coding                        22895
lncRNA                                20062
processed_pseudogene                  10804
unprocessed_pseudogene                 3340
misc_RNA                               2407
snRNA                                  2072
miRNA                                  1924
transcribed_unprocessed_pseudogene     1130
TEC                                    1111
snoRNA                                 1009
Name: gene_biotype, dtype: int64

In [13]:
ensg.gene_df.seq_region_exc_type.value_counts(dropna=False)

None           62703
HAP             4816
PATCH_FIX       1212
PATCH_NOVEL      561
Name: seq_region_exc_type, dtype: int64

In [14]:
ensg.gene_df.mhc.value_counts()

no      66632
MHC      2468
xMHC      192
Name: mhc, dtype: int64

In [15]:
len(ensg.gene_df)

69292

## alternative gene alleles

Related:

- [OTP: Origin of genes_with_non_reference_ensembl_ids.tsv](https://github.com/opentargets/platform/issues/702)
- [biostars: map between different assemblies of one ensembl release](https://www.biostars.org/p/143956/)
- using `attrib_type.code = "non_ref"` for `primary_assembly` doesn't appear to return any results

In [16]:
ensg.alt_allele_df.head()

,ensembl_gene_id,alt_allele_group_id,alt_allele_is_representative,primary_assembly,seq_region,alt_allele_attrib,ensembl_created_date,ensembl_representative_gene_id,is_representative_gene,representative_gene_method
0,ENSG00000282572,44429,False,True,7,AUTOMATICALLY_ASSIGNED,2015-06-01 18:57:05,ENSG00000282572,True,primary_assembly
1,ENSG00000281951,44429,False,False,CHR_HSCHR7_2_CTG1,AUTOMATICALLY_ASSIGNED,2015-06-01 18:57:05,ENSG00000282572,False,primary_assembly
2,ENSG00000273644,44430,False,True,7,AUTOMATICALLY_ASSIGNED,2014-06-09 10:49:07,ENSG00000273644,True,primary_assembly
3,ENSG00000282333,44430,False,False,CHR_HSCHR7_2_CTG1,AUTOMATICALLY_ASSIGNED,2015-06-01 18:57:05,ENSG00000273644,False,primary_assembly
4,ENSG00000232325,44431,False,True,7,AUTOMATICALLY_ASSIGNED,2009-05-19 09:47:17,ENSG00000232325,True,primary_assembly


In [17]:
# looks like non_ref isn't set for human genes
query = '''
SELECT *
FROM gene_attrib
LEFT JOIN attrib_type
  ON gene_attrib.attrib_type_id = attrib_type.attrib_type_id
WHERE attrib_type.code = "non_ref"
LIMIT 5
'''
pd.read_sql(sql=query, con=ensg.connection_url)

,gene_id,attrib_type_id,value,attrib_type_id,code,name,description


In [18]:
ensg.alt_allele_df.alt_allele_attrib.value_counts()

AUTOMATICALLY_ASSIGNED    7795
IS_REPRESENTATIVE         1291
Name: alt_allele_attrib, dtype: int64

In [19]:
ensg.alt_allele_df.query("is_representative_gene").representative_gene_method.value_counts()

primary_assembly                1950
alt_allele_is_representative    1291
first_added                       37
Name: representative_gene_method, dtype: int64

In [20]:
ensg.gene_df.query("ensembl_gene_id != ensembl_representative_gene_id").head(2)

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
764,ENSG00000056678,11,KIFC1,HGNC,HGNC:6389,lncRNA,havana,2008-04-29 11:17:41,2010-03-18 16:07:21,GRCh38,...,33561891,33562220,1,False,NaN,no,kinesin family member C1,HGNC Symbol,HGNC:6389,ENSG00000237649
2059,ENSG00000096150,9,RPS18,HGNC,HGNC:10401,protein_coding,ensembl_havana,2008-04-29 11:17:41,2015-06-01 18:57:05,GRCh38,...,33200834,33205337,1,False,NaN,MHC,ribosomal protein S18,HGNC Symbol,HGNC:10401,ENSG00000231500


# replaced ID converter

A single `old_stable_id` can map to multiple `new_stable_id`. For example, `ENSG00000152006`

https://uswest.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=AzhM62SpkvdiLC4H-6808613

Requested ID | Matched ID(s) | Releases
-- | -- | --
ENSG00000152006 | ENSG00000196273 | 26: ENSG00000196273.1
ENSG00000152006 | ENSG00000197016 | 26: ENSG00000197016.1
ENSG00000152006 | ENSG00000196239 | 26: ENSG00000196239.1

In [21]:
ensg.old_to_new_df.head(2)

,old_ensembl_gene_id,new_ensembl_gene_id
0,ENSG00000011319,ENSG00000187391
1,ENSG00000077754,ENSG00000197226


In [22]:
# some ensembl genes replaced by many new ensembl genes
ensg.old_to_new_df.old_ensembl_gene_id.value_counts().head(2)

ENSG00000201456    91
ENSG00000193147    90
Name: old_ensembl_gene_id, dtype: int64

In [23]:
# example
ensg._update_ensembl_gene("ENSG00000152006")

{'ENSG00000196239', 'ENSG00000196273', 'ENSG00000197016'}

In [24]:
ensg.old_to_newest_df.head(2)

,old_ensembl_gene_id,newest_ensembl_gene_id,is_current
0,ASMPATCHG00000000170,ENSG00000256229,True
1,ASMPATCHG00000000174,ENSG00000188171,True


In [25]:
len(ensg.old_to_newest_df)

19365

In [26]:
ensg.old_to_newest_df.is_current.value_counts()

True     10331
False     9034
Name: is_current, dtype: int64

## omni-updater

The omni-updater dataset is designed to convert ensembl gene IDs from input data to the current, representative ensembl_gene_ids for this ensembl release. It assumes:

- users want to update outdated genes with their replacements
- users want a dataset of representative genes only, and want to convert alternative alleles to representative genes

An inner join of a dataset with `update_df` on `input_ensembl_gene_id` will do the following:

- produce output ensembl_gene_ids that are current and representatives
- update outdated genes with their current identifiers. Outdated genes with no current replacement will be removed by the inner join.
- update alternative gene alleles with their representatives
- genes that are already represenative and current will map to themselves

In [27]:
ensg.update_df.head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
0,ASMPATCHG00000000170,ENSG00000256229,False,True,1,3
2,ASMPATCHG00000000174,ENSG00000188171,False,True,1,3


In [28]:
ensg.update_df.sort_values("input_maps_to_n_genes", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
4005,ENSG00000201456,ENSG00000206708,False,True,90,6
3725,ENSG00000201456,ENSG00000206996,False,True,90,7


In [29]:
ensg.update_df.sort_values("n_inputs_map_to_gene", ascending=False).head(2)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
2208,ENSG00000188767,ENSG00000205936,False,True,1,58
2246,ENSG00000198061,ENSG00000205936,False,True,2,58


In [30]:
(ensg.update_df.input_maps_to_n_genes == 1).mean()

0.9179142349657448

In [31]:
ensg.update_df.query("ensembl_gene_id == 'ENSG00000256263'")

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
633,ASMPATCHG00000001513,ENSG00000256263,False,True,1,13
44967,ENSG00000256263,ENSG00000256263,True,True,1,13
634,ENSG00000260270,ENSG00000256263,False,True,1,13
635,ENSG00000262195,ENSG00000256263,False,True,1,13
636,ENSG00000263289,ENSG00000256263,False,True,1,13
637,ENSG00000269129,ENSG00000256263,False,True,1,13
638,ENSG00000269829,ENSG00000256263,False,True,1,13
639,ENSG00000270320,ENSG00000256263,False,True,1,13
640,ENSG00000271301,ENSG00000256263,False,True,1,13
641,ENSG00000271747,ENSG00000256263,False,True,1,13


In [32]:
print(
    f"The omni-updater contains {len(ensg.update_df):,} rows for mapping "
    f"{ensg.update_df.input_ensembl_gene_id.nunique():,} input genes to "
    f"{ensg.update_df.ensembl_gene_id.nunique():,} current, representative genes."
)

The omni-updater contains 78,820 rows for mapping 73,137 input genes to 63,484 current, representative genes.


In [33]:
# https://useast.ensembl.org/Homo_sapiens/Tools/IDMapper/Results?tl=P45VLMbogubpI0QA-6815464
ensg.update_df.query("input_ensembl_gene_id == 'ENSG00000201456'").head(3)

,input_ensembl_gene_id,ensembl_gene_id,input_current,input_representative,input_maps_to_n_genes,n_inputs_map_to_gene
5676,ENSG00000201456,ENSG00000199482,False,True,90,2
3452,ENSG00000201456,ENSG00000199601,False,True,90,7
5677,ENSG00000201456,ENSG00000199664,False,True,90,3


## cross-refrences (xrefs)

In [34]:
ensg.xref_df.head()

,ensembl_representative_gene_id,ensembl_gene_id,gene_symbol,xref_source,xref_accession,xref_label,xref_description,xref_info_type,xref_linkage_annotation,xref_curie
0,ENSG00000000003,ENSG00000000003,TSPAN6,ArrayExpress,ENSG00000000003,ENSG00000000003,None,DIRECT,None,arrayexpress:ENSG00000000003
1,ENSG00000000003,ENSG00000000003,TSPAN6,EntrezGene,7105,TSPAN6,tetraspanin 6,DEPENDENT,None,ncbigene:7105
2,ENSG00000000003,ENSG00000000003,TSPAN6,HGNC,HGNC:11858,TSPAN6,tetraspanin 6,DIRECT,None,hgnc:11858
3,ENSG00000000003,ENSG00000000003,TSPAN6,MIM_GENE,300191,TETRASPANIN 6; TSPAN6 [*300191],TETRASPANIN 6; TSPAN6;;TRANSMEMBRANE 4 SUPERFA...,DEPENDENT,None,omim:300191
4,ENSG00000000003,ENSG00000000003,TSPAN6,Uniprot_gn,A0A087WYV6,TSPAN6,None,DEPENDENT,None,uniprot:A0A087WYV6


In [35]:
# datasets where there are ensembl_gene_id-xref_source-xref_accession pairs might not be distinct 
xref_dup_df = ensg.xref_df[ensg.xref_df.duplicated(subset=["ensembl_gene_id", "xref_source", "xref_accession"], keep=False)]
xref_dup_df.xref_source.value_counts()

HGNC    20
Name: xref_source, dtype: int64

In [36]:
# xref sources versus info_types
df = pd.crosstab(ensg.xref_df.xref_source, ensg.xref_df.xref_info_type, margins=True)
df["bioregistry_prefix"] = df.index.to_series().replace(ensg._xref_prefix_updater).map(normalize_prefix)
df

xref_info_type,DEPENDENT,DIRECT,MISC,All,bioregistry_prefix
xref_source,,,,,
ArrayExpress,0,69292,0,69292,arrayexpress
DBASS3,0,19,0,19,None
DBASS5,0,14,0,14,None
ENS_LRG_gene,0,1324,0,1324,lrg
EntrezGene,35733,0,0,35733,ncbigene
HGNC,1604,43460,0,45064,hgnc
MIM_GENE,18501,0,0,18501,omim
MIM_MORBID,7757,0,0,7757,omim
RFAM,0,5080,0,5080,rfam


## Gene Ontology xrefs

In [37]:
ensg.xref_go_df.head(3)

,ensembl_gene_id,go_id,go_label,go_evidence_codes,xref_info_types,xref_info_texts,ensembl_transcript_ids,ensembl_representative_gene_id
0,ENSG00000000003,GO:0005515,protein binding,IPI,DIRECT,UniProt,ENST00000373020,ENSG00000000003
1,ENSG00000000003,GO:0005887,integral component of plasma membrane,IBA,DIRECT,GO_Central,ENST00000373020,ENSG00000000003
2,ENSG00000000003,GO:0016020,membrane,IEA,DIRECT,UniProt,"ENST00000373020,ENST00000612152,ENST00000614008",ENSG00000000003


In [38]:
# GO terms for CCR5
# compare to http://useast.ensembl.org/Homo_sapiens/Gene/Ontologies/molecular_function?g=ENSG00000160791
sorted(ensg.xref_go_df.query("ensembl_gene_id == 'ENSG00000160791'").go_label)

['C-C chemokine binding',
 'C-C chemokine receptor activity',
 'G protein-coupled receptor activity',
 'G protein-coupled receptor signaling pathway',
 'MAPK cascade',
 'actin binding',
 'calcium ion transport',
 'calcium-mediated signaling',
 'cell chemotaxis',
 'cell surface',
 'cell surface receptor signaling pathway',
 'cell-cell signaling',
 'cellular defense response',
 'cellular response to lipopolysaccharide',
 'chemokine (C-C motif) ligand 5 binding',
 'chemokine receptor activity',
 'chemokine-mediated signaling pathway',
 'chemotaxis',
 'coreceptor activity',
 'cytoplasm',
 'defense response',
 'dendritic cell chemotaxis',
 'endosome',
 'external side of plasma membrane',
 'identical protein binding',
 'immune response',
 'inflammatory response',
 'integral component of membrane',
 'integral component of plasma membrane',
 'membrane',
 'negative regulation of macrophage apoptotic process',
 'phosphatidylinositol phospholipase C activity',
 'plasma membrane',
 'positive regul

## lrg xrefs

In [39]:
ensg.xref_lrg_df.head(2)

,ensembl_gene_id,lrg_gene_id
83,ENSG00000000971,LRG_47
113,ENSG00000001084,LRG_1166


In [40]:
len(ensg.xref_lrg_df)

1324

### ncbigene xrefs

In [41]:
ensg.xref_ncbigene_df.head()

,ensembl_representative_gene_id,ncbigene_id,gene_symbol,ncbigene_symbol
1,ENSG00000000003,7105,TSPAN6,TSPAN6
9,ENSG00000000005,64102,TNMD,TNMD
15,ENSG00000000419,8813,DPM1,DPM1
43,ENSG00000000457,57147,SCYL3,SCYL3
50,ENSG00000000460,55732,C1orf112,C1orf112


In [42]:
# ensembl gene mapped to by multiple ncbigenes
ensg.xref_ncbigene_df.ensembl_representative_gene_id.value_counts().head(3)

ENSG00000000003    1
ENSG00000204511    1
ENSG00000204711    1
Name: ensembl_representative_gene_id, dtype: int64

In [43]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ensembl_representative_gene_id.duplicated().sum()

(26739, 0)

In [44]:
# ncbigene mapped to by multiple ensembl genes, likely due to alt gene alleles
ensg.xref_ncbigene_df.ncbigene_id.value_counts().head(3)

124905408    21
124904108    12
124904135    10
Name: ncbigene_id, dtype: int64

In [45]:
len(ensg.xref_ncbigene_df), ensg.xref_ncbigene_df.ncbigene_id.duplicated().sum()

(26739, 196)

In [46]:
# ensg.xref_ncbigene_df.query("ensembl_representative_gene_id == 'ENSG00000231500'")
# ensg.xref_ncbigene_df.query("ncbigene_id == '51206'")

In [47]:
repr_ensembl_gene_ids = set(ensg.gene_df.ensembl_representative_gene_id)
len(repr_ensembl_gene_ids)

63484

In [48]:
# many of these genes should probably be alternative alleles rather than representative
ensg.gene_df.query("not primary_assembly and ensembl_gene_id==ensembl_representative_gene_id")

,ensembl_gene_id,ensembl_gene_version,gene_symbol,gene_symbol_source_db,gene_symbol_source_id,gene_biotype,ensembl_source,ensembl_created_date,ensembl_modified_date,coord_system_version,...,seq_region_start,seq_region_end,seq_region_strand,primary_assembly,lrg_gene_id,mhc,gene_description,gene_description_source_db,gene_description_source_id,ensembl_representative_gene_id
16989,ENSG00000196101,9,HLA-DRB3,HGNC,HGNC:4951,protein_coding,ensembl_havana,2008-04-29 11:17:41,2016-06-03 16:05:49,GRCh38,...,32449765,32462852,-1,False,NaN,MHC,"major histocompatibility complex, class II, DR...",HGNC Symbol,HGNC:4951,ENSG00000196101
23346,ENSG00000215523,7,ENSG00000215523,None,None,lncRNA,havana,2007-09-07 00:01:32,2015-06-01 18:57:05,GRCh38,...,30788147,30820172,-1,False,NaN,MHC,chromosome 6 open reading frame 214 (putative),NaN,NaN,ENSG00000215523
24797,ENSG00000223359,2,ENSG00000223359,None,None,unprocessed_pseudogene,havana,2009-05-19 09:47:17,2010-03-18 16:07:21,GRCh38,...,32524368,32524463,-1,False,NaN,MHC,"major histocompatibility complex, class II, DR...",NaN,NaN,ENSG00000223359
24940,ENSG00000223557,1,ENSG00000223557,None,None,lncRNA,havana,2009-05-19 09:47:17,2009-05-19 09:47:17,GRCh38,...,31871305,31871598,1,False,NaN,MHC,novel transcript,NaN,NaN,ENSG00000223557
25167,ENSG00000223844,1,ENSG00000223844,None,None,processed_pseudogene,havana,2009-05-19 09:47:17,2009-05-19 09:47:17,GRCh38,...,32682668,32683798,1,False,NaN,MHC,"family with sequence similarity 8, member A5 p...",NaN,NaN,ENSG00000223844
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68631,ENSG00000290638,1,ENSG00000290638,None,None,lncRNA,havana,2022-03-02 14:40:18,2022-03-02 14:40:18,GRCh38,...,18441656,18452500,1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000290638
68632,ENSG00000290639,1,ENSG00000290639,None,None,lncRNA,havana,2022-03-02 14:40:19,2022-03-02 14:40:19,GRCh38,...,18506700,18515918,-1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000290639
68633,ENSG00000290640,1,ENSG00000290640,None,None,lncRNA,havana,2022-03-02 14:40:20,2022-03-02 14:40:20,GRCh38,...,75550334,75579999,-1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000290640
69152,ENSG00000291160,1,ENSG00000291160,None,None,lncRNA,havana,2022-03-02 14:52:31,2022-03-02 14:52:31,GRCh38,...,82298669,82305817,1,False,NaN,no,novel transcript,NaN,NaN,ENSG00000291160
